In [ ]:
# !pip install plotly kaleido datasets nbformat -U -q

In [1]:
import os

import datasets
import pandas as pd
from dotenv import load_dotenv
from huggingface_hub import login


load_dotenv(override=True)
login(os.getenv("HF_TOKEN"))

pd.set_option("max_colwidth", None)

OUTPUT_DIR = "../../output"

/Users/aymeric/venv/agents/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
eval_ds = datasets.load_dataset("gaia-benchmark/GAIA", "2023_all")["validation"]
eval_ds = eval_ds.rename_columns({"Question": "question", "Final answer": "true_answer", "Level": "task"})
eval_df = pd.DataFrame(eval_ds)

Using the latest cached version of the module from /Users/aymeric/.cache/huggingface/modules/datasets_modules/datasets/gaia-benchmark--GAIA/ec492fe4320ee795b1aed6bb46229c5f693226b0f1316347501c24b4baeee005 (last modified on Tue May 28 10:04:32 2024) since it couldn't be found locally at gaia-benchmark/GAIA, or remotely on the Hugging Face Hub.


# 1. Load all results

In [3]:
import glob


results = []
for f in glob.glob(f"{OUTPUT_DIR}/validation/*.jsonl"):
    df = pd.read_json(f, lines=True)
    df["agent_name"] = f.split("/")[-1].split(".")[0]
    results.append(df)

result_df = pd.concat(results)
result_df["prediction"] = result_df["prediction"].fillna("No prediction")

In [4]:
import re
from collections import Counter

from scripts.gaia_scorer import check_close_call, question_scorer


result_df["is_correct"] = result_df.apply(lambda x: question_scorer(x["prediction"], x["true_answer"]), axis=1)
result_df["is_near_correct"] = result_df.apply(
    lambda x: check_close_call(x["prediction"], x["true_answer"], x["is_correct"]),
    axis=1,
)

result_df["count_steps"] = result_df["intermediate_steps"].apply(len)


def find_attachment(question):
    matches = eval_df.loc[eval_df["question"].apply(lambda x: x in question), "file_name"]

    if len(matches) == 0:
        return "Not found"
    file_path = matches.values[0]

    if isinstance(file_path, str) and len(file_path) > 0:
        return file_path.split(".")[-1]
    else:
        return "None"


result_df["attachment_type"] = result_df["question"].apply(find_attachment)


def extract_tool_calls(code):
    regex = r"\b(\w+)\("
    function_calls = [el for el in re.findall(regex, code) if el.islower()]

    function_call_counter = Counter(function_calls)
    return function_call_counter


def sum_tool_calls(steps):
    total_count = Counter()
    for step in steps:
        if "llm_output" in step:
            total_count += extract_tool_calls(step["llm_output"])

    return total_count


def get_durations(row):
    # start_datetime = datetime.strptime(row['start_time'], "%Y-%m-%d %H:%M:%S")
    # end_datetime = datetime.strptime(row['end_time'], "%Y-%m-%d %H:%M:%S")

    duration_timedelta = row["end_time"] - row["start_time"]
    return int(duration_timedelta.total_seconds())


result_df["duration"] = result_df.apply(get_durations, axis=1)
# result_df["tool_calls"] = result_df["intermediate_steps"].apply(sum_tool_calls)

/Users/aymeric/Documents/Code/smolagents/examples/open_deep_research/scripts/gaia_scorer.py:52: UserWarning: Answer lists have different lengths, returning False.
  warnings.warn("Answer lists have different lengths, returning False.", UserWarning)


String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to number str.
String Unable to determine cannot be normalized to numb

In [5]:
def get_thoughts(x):
    try:
        output = x[0]["task"]
        for y in x[1:]:
            try:
                if "observation" in y:
                    output += y["llm_output"] + "\nObservation:" + y["observation"]
                else:
                    output += y["llm_output"] + r"\Error:" + str(y["error"])
            except Exception:
                pass
        return output
    except Exception:
        return None


result_df["thoughts"] = result_df["intermediate_steps"].apply(lambda x: get_thoughts(x))

In [6]:
result_df["agent_name"].value_counts()

agent_name
07-apr-refacto_system_prompt_noplanningfacts         165
generate-traces-03-apr-noplanning                    165
07-apr-refacto_system_prompt_noplanning              165
07-apr-refacto_system_prompt_noplanning_qwen         165
8-apr_refacto-system-prompt-nosummary                165
07-apr-refacto_system_prompt_trueplanningnofacts     164
07-apr-refacto_system_prompt_trueplanningfacts       164
generate-traces-03-apr                               164
07-apr-refacto_system_prompt                         163
o3-18-apr                                            163
07-apr-refacto_system_prompt_trueplanningnewfacts    145
Name: count, dtype: int64

# 2. Inspect specific runs

In [7]:
sel_df = result_df
# sel_df = sel_df.loc[
#     (result_df["agent_name"].isin(list_versions))
# ]
sel_df = sel_df.reset_index(drop=True)
display(sel_df["agent_name"].value_counts())
sel_df = sel_df.drop_duplicates(subset=["agent_name", "question"])
display(sel_df.groupby("agent_name")[["task"]].value_counts())
print("Total length:", len(sel_df), "- is complete:", len(sel_df) == 165)

agent_name
07-apr-refacto_system_prompt_noplanningfacts         165
generate-traces-03-apr-noplanning                    165
07-apr-refacto_system_prompt_noplanning              165
07-apr-refacto_system_prompt_noplanning_qwen         165
8-apr_refacto-system-prompt-nosummary                165
07-apr-refacto_system_prompt_trueplanningnofacts     164
07-apr-refacto_system_prompt_trueplanningfacts       164
generate-traces-03-apr                               164
07-apr-refacto_system_prompt                         163
o3-18-apr                                            163
07-apr-refacto_system_prompt_trueplanningnewfacts    145
Name: count, dtype: int64

agent_name                                         task
07-apr-refacto_system_prompt                       2       84
                                                   1       53
                                                   3       26
07-apr-refacto_system_prompt_noplanning            2       86
                                                   1       53
                                                   3       26
07-apr-refacto_system_prompt_noplanning_qwen       2       86
                                                   1       53
                                                   3       26
07-apr-refacto_system_prompt_noplanningfacts       2       86
                                                   1       53
                                                   3       26
07-apr-refacto_system_prompt_trueplanningfacts     2       85
                                                   1       53
                                                   3       26
07-apr-refacto

Total length: 1788 - is complete: False


In [8]:
display("Average score:", sel_df.groupby("agent_name")[["is_correct"]].mean().round(3))
display(
    sel_df.groupby(["agent_name", "task"])[["is_correct", "is_near_correct", "count_steps", "question", "duration"]]
    .agg(
        {
            "is_correct": "mean",
            "is_near_correct": "mean",
            "count_steps": "mean",
            "question": "count",
            "duration": "mean",
        }
    )
    .rename(columns={"question": "count"})
)

'Average score:'

,is_correct
agent_name,
07-apr-refacto_system_prompt,0.356
07-apr-refacto_system_prompt_noplanning,0.358
07-apr-refacto_system_prompt_noplanning_qwen,0.176
07-apr-refacto_system_prompt_noplanningfacts,0.424
07-apr-refacto_system_prompt_trueplanningfacts,0.409
07-apr-refacto_system_prompt_trueplanningnewfacts,0.393
07-apr-refacto_system_prompt_trueplanningnofacts,0.390
8-apr_refacto-system-prompt-nosummary,0.430
generate-traces-03-apr,0.360


is_correct  \
agent_name                                        task               
07-apr-refacto_system_prompt                      1       0.471698   
                                                  2       0.345238   
                                                  3       0.153846   
07-apr-refacto_system_prompt_noplanning           1       0.528302   
                                                  2       0.313953   
                                                  3       0.153846   
07-apr-refacto_system_prompt_noplanning_qwen      1       0.264151   
                                                  2       0.174419   
                                                  3       0.000000   
07-apr-refacto_system_prompt_noplanningfacts      1       0.471698   
                                                  2       0.465116   
                                                  3       0.192308   
07-apr-refacto_system_prompt_trueplanningfacts    1       0.471698   
                                                  2       0.423529   
                                                  3       0.230769   
07-apr-refacto_system_prompt_trueplanningnewfacts 1       0.458333   
                                                  2       0.434211   
                                                  3       0.095238   
07-apr-refacto_system_prompt_trueplanningnofacts  1       0.490566   
                                                  2       0.400000   
                                                  3       0.153846   
8-apr_refacto-system-prompt-nosummary             1       0.509434   
                                                  2       0.441860   
                                                  3       0.230769   
generate-traces-03-apr                            1       0.415094   
                                                  2       0.395349   
                                                  3       0.120000   
generate-traces-03-apr-noplanning                 1       0.452830   
                                                  2       0.418605   
                                                  3       0.192308   
o3-18-apr                                         1       0.679245   
                                                  2       0.588235   
                                                  3       0.360000   

                                                        is_near_correct  \
agent_name                                        task                    
07-apr-refacto_system_prompt                      1            0.509434   
                                                  2            0.357143   
                                                  3            0.153846   
07-apr-refacto_system_prompt_noplanning           1            0.547170   
                                                  2            0.325581   
                                                  3            0.153846   
07-apr-refacto_system_prompt_noplanning_qwen      1            0.264151   
                                                  2            0.174419   
                                                  3            0.000000   
07-apr-refacto_system_prompt_noplanningfacts      1            0.509434   
                                                  2            0.476744   
                                                  3            0.192308   
07-apr-refacto_system_prompt_trueplanningfacts    1            0.471698   
                                                  2            0.435294   
                                                  3            0.230769   
07-apr-refacto_system_prompt_trueplanningnewfacts 1            0.479167   
                                                  2            0.434211   
                                                  3            0.095238   
07-apr-refacto_system_prompt_trueplanningnofacts  1            0.490566   
                              

In [12]:
import plotly.express as px


cumulative_df = (
    (
        sel_df.groupby("agent_name")[["is_correct", "is_near_correct"]]
        .expanding(min_periods=1, axis=0, method="single")
        .agg({"is_correct": "mean", "is_near_correct": "count"})
        .reset_index()
    )
    .copy()
    .rename(columns={"is_near_correct": "index"})
)
cumulative_df["index"] = cumulative_df["index"].astype(int) - 1


def find_question(row):
    try:
        res = sel_df.loc[sel_df["agent_name"] == row["agent_name"], "question"].iloc[row["index"]][:50]
        return res
    except Exception:
        return ""


cumulative_df["question"] = cumulative_df.apply(find_question, axis=1)

px.line(
    cumulative_df,
    color="agent_name",
    x="index",
    y="is_correct",
    hover_data="question",
)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([['An office held a Secret Santa gift exchange where '],
                                   ["Here's a fun riddle that I think you'll enjoy.\n\nYo"],
                                   ['When you take the average of the standard populati'],
                                   ['The attached spreadsheet shows the inventory for a'],
                                   ['.rewsna eht sa "tfel" drow eht fo etisoppo eht eti'],
                                   ['Using the Biopython library in Python, parse the P'],
                                   ['What was the volume in m^3 of the fish bag that wa'],
                                   ['In Series 9, Episode 11 of Doctor Who, the Doctor '],
                                   ['How many studio albums were published by Mercedes '],
                                   ['If we assume all articles published by Nature in 2'],
                                   ['What two-word type of model did Manash Pratim Kash'],
                                   ['A paper about AI regulation that was originally su'],
                                   ['In Unlambda, what exact charcter or text needs to '],
                                   ["The object in the British Museum's collection with"],
                                   ['Each cell in the attached spreadsheet represents a'],
                                   ['Use density measures from the chemistry materials '],
                                   ['¬(A ∧ B) ↔ (¬A ∨ ¬B)\n¬(A ∨ B) ↔ (¬A ∧ ¬B)\n(A → B) '],
                                   ['My family reunion is this week, and I was assigned'],
                                   ['Assuming scientists in the famous youtube video Th'],
                                   ['If Eliud Kipchoge could maintain his record-making'],
                                   ['I’m researching species that became invasive after'],
                                   ['In the fictional language of Tizin, basic sentence'],
                                   ['The photograph in the Whitney Museum of American A'],
                                   ['Compute the check digit the Tropicos ID for the Or'],
                                   ['What integer-rounded percentage of the total lengt'],
                                   ['How many High Energy Physics - Lattice articles li'],
                                   ['How many applicants for the job in the PDF are onl'],
                                   ['In the video https://www.youtube.com/watch?v=L1vXC'],
                                   ['What is the average number of pre-2020 works on th'],
                                   ['What are the EC numbers of the two most commonly u'],
                                   ['Review the chess position provided in the image. I'],
                                   ['Which of the text elements under CATEGORIES in the'],
                                   ["Under DDC 633 on Bielefeld University Library's BA"],
                                   ['The attached file contains a list of vendors in th'],
                                   ['In April of 1977, who was the Prime Minister of th'],
                                   ["What's the last line of the rhyme under the flavor"],
                                   ['Of the authors (First M. Last) that worked on the '],
                                   ['Given this table defining * on the set S = {a, b, '],
                                   ['I need to fact-check a citation. This is the citat'],
                                   ["In Emily Midkiff's June 2014 article in a journal "],
                                   ['The attached file shows a list of books in the col'],
                                   ['What writer is quoted by Merriam-Webster for the W'],
                                   ['As a comma separated list with no whitespace, usin'],
                                   ['Who nominated the only Featured Artic

# 3. Dive deeper into one run

In [ ]:
sel_df = result_df.loc[result_df["agent_name"] == "o1"]
print(len(sel_df))

### Count errors

In [ ]:
import numpy as np


error_types = [
    "AgentParsingError",
    "AgentExecutionError",
    "AgentMaxIterationsError",
    "AgentGenerationError",
]
sel_df[error_types] = 0
sel_df["Count steps"] = np.nan


def count_errors(row):
    if isinstance(row["intermediate_steps"], list):
        row["Count steps"] = len(row["intermediate_steps"])
        for step in row["intermediate_steps"]:
            if isinstance(step, dict) and "error" in step:
                try:
                    row[str(step["error"]["error_type"])] += 1
                except Exception:
                    pass
    return row


sel_df = sel_df.apply(count_errors, axis=1)

In [ ]:
import plotly.express as px


aggregate_errors = (
    sel_df.groupby(["is_correct"])[error_types + ["Count steps"]].mean().reset_index().melt(id_vars=["is_correct"])
)

fig = px.bar(
    aggregate_errors,
    y="value",
    x="variable",
    color="is_correct",
    labels={
        "agent_name": "<b>Model</b>",
        "task": "<b>Level</b>",
        "aggregate_score": "<b>Performance</b>",
        "value": "<b>Average count</b>",
        "eval_score_GPT4": "<b>Score</b>",
    },
)
fig.update_layout(
    height=500,
    width=800,
    barmode="group",
    bargroupgap=0.0,
)
fig.update_traces(textposition="outside")
fig.write_image("aggregate_errors.png", scale=3)
fig.show()

### Inspect result by file extension type

In [ ]:
display(
    result_df.groupby(["attachment_type"])[["is_correct", "count_steps", "question"]].agg(
        {"is_correct": "mean", "count_steps": "mean", "question": "count"}
    )
)

# 4. Ensembling methods

In [ ]:
counts = result_df["agent_name"].value_counts()
long_series = result_df.loc[result_df["agent_name"].isin(counts[counts > 140].index)]

In [ ]:
def majority_vote(df):
    df = df[(df["prediction"] != "Unable to determine") & (~df["prediction"].isna()) & (df["prediction"] != "None")]

    answer_modes = df.groupby("question")["prediction"].agg(lambda x: x.mode()[0]).reset_index()
    first_occurrences = (
        df.groupby(["question", "prediction"]).agg({"task": "first", "is_correct": "first"}).reset_index()
    )
    result = answer_modes.merge(first_occurrences, on=["question", "prediction"], how="left")

    return result


def oracle(df):
    def get_first_correct_or_first_wrong(group):
        correct_answers = group[group["is_correct"]]
        if len(correct_answers) > 0:
            return correct_answers.iloc[0]
        return group.iloc[0]

    result = df.groupby("question").apply(get_first_correct_or_first_wrong)

    return result.reset_index(drop=True)


display((long_series.groupby("agent_name")["is_correct"].mean() * 100).round(2))
print(f"Majority score: {majority_vote(long_series)['is_correct'].mean() * 100:.2f}")
print(f"Oracle score: {oracle(long_series)['is_correct'].mean() * 100:.2f}")

### Submit

In [ ]:
agent_run = "code_o1_04_february_submission5.jsonl"
df = pd.read_json(f"output/validation/{agent_run}", lines=True)
df = df[["task_id", "prediction", "intermediate_steps"]]
df = df.rename(columns={"prediction": "model_answer", "intermediate_steps": "reasoning_trace"})

In [ ]:
df.to_json("submission.jsonl", orient="records", lines=True)